# 学習中に正則化を強める

物性値は
$\theta_F = \pi/100$,$\eta_F=1\degree$とする。

In [ ]:
#------------------------
# Google Colab上でのみ実行
#------------------------
import time
%env TOKEN=*************************************
! git clone https://$$TOKEN@github.com/konnitiha3/MOD2NN.git

import sys
sys.path.append('/content/MOD2NN')

from google.colab import drive
drive.mount('/content/drive')

In [2]:
import time
import os
import json
import sys
import re
import glob
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from Faraday.two_dim.module.lib.layers import *
from Faraday.two_dim.module.lib import regularizer

tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.8.0
Python: 3.8.11 (default, Aug 16 2021, 12:04:33) 
[Clang 12.0.0 ]


In [13]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-5000:]
y_val = y_train[-5000:]
x_train = x_train[:-5000]
y_train = y_train[:-5000]

ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

ds_train_batch = ds_train.batch(64)

In [8]:
#@title デフォルトのタイトル テキスト
wavelength = 633.0e-9 #@param {type:"number"}
d = 1.0e-6 #@param {type:"number"}
n = 1.5 #@param {type:"number"}
z = 0.7e-3 #@param {type:"number"}

In [23]:
def create_model(**kwargs):
    shape = (100, 100)
    inputs = tf.keras.Input((28, 28))
    theta = np.pi / 100
    eta = np.tan(1.0*np.pi/100)
    l1 = kwargs["l1"]
    tf.random.set_seed(kwargs["seed"])
    x = ImageResizing(shape)(inputs)
    x = IntensityToElectricField(shape)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand')(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta, kernel_regularizer=regularizer.SymmetricShiftL1Regularizer(l1, np.pi/2))(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand')(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta, kernel_regularizer=regularizer.SymmetricShiftL1Regularizer(l1, np.pi/2))(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand')(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta, kernel_regularizer=regularizer.SymmetricShiftL1Regularizer(l1, np.pi/2))(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand')(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta, kernel_regularizer=regularizer.SymmetricShiftL1Regularizer(l1, np.pi/2))(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand')(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta, kernel_regularizer=regularizer.SymmetricShiftL1Regularizer(l1, np.pi/2))(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand')(x)
    x = FaradayRotationByStokes(shape)(x)
    x = MNISTDetector(10)(x)
    x = tf.keras.layers.Softmax()(x)
    model = tf.keras.Model(inputs, x)
    return model

In [31]:
def test_model():
    inputs = tf.keras.Input((28, 28))
    x = tf.keras.layers.Flatten(input_shape=(28,28))(inputs)
    x = tf.keras.layers.Dense(128)(x)
    x = tf.keras.layers.Dense(10)(x)
    x = tf.keras.layers.Softmax()(x)

    return tf.keras.Model(inputs, x)

In [41]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(lr=0.001)

In [25]:
def loss(model, x, y, training):
    y_ = model(x, training=training)

    return loss_object(y_true=y, y_pred=y_)

In [ ]:
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [51]:
def train(name, seed):
    tf.random.set_seed(seed)

    num_epochs = 50
    l1_rate = 5

    a1 = 0.0
    a2 = 1.0e-9
    r = 10.0 # 等比級数

    model = create_model(l1=a1) #　最初は0
    history = []


    model_name = name
    cholab_path = "/content/drive/MyDrive/D2NN/"

    for epoch in range(num_epochs):
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

        # l1_rateごとにl1を変える
        if epoch % l1_rate == 0:
            weights = model.get_weights()
            l1 = a2*r**(epoch/5) #等比数列
            model = create_model(l1=l1)
            model.set_weights(weights)

        # batch train

        for x, y, in ds_train_batch:
            loss_value, grads = grad(model, x, y)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            epoch_loss_avg.update_state(loss_value)
            epoch_accuracy.update_state(y, model(x, training=True))

        # End Epoch

        val_loss = loss(model, x_val, y_val, training=False)

        val_accuracy.update_state(y_val, model(x_val, training=False))

        history.append([epoch,
            epoch_loss_avg.result().numpy(),
            epoch_accuracy.result().numpy(),
            val_loss.numpy(),
            val_accuracy.result().numpy()])

        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}, Val Loss: {:.3f}, Val Accuracy: {:.3f}".format(
            epoch,
            epoch_loss_avg.result(),
            epoch_accuracy.result(),
            val_loss.numpy(),
            val_accuracy.result()
        ))
        checkpoint_path = cholab_path + "checkpoint/" + model_name + "/cp-epoch:{:04d}.ckpt".format(epoch)
        model.save_weights(checkpoint_path, save_format="tf")

        print(history)

    path = cholab_path + "trained_model/"+ model_name
    model.save(path)

    df = pd.DataFrame(history, columns=["epoch", "loss", "accuracy", "val_loss", "val_accuracy"])
    df.to_csv(path + "/history.csv")

    with open(path + "/config.json", 'w') as f:
        json.dump(model.get_config(), f, indent=4)

Epoch 000: Loss: 0.277, Accuracy: 93.180%, Val Loss: 0.247, Val Accuracy: 0.937
[[0, 0.2766259, 0.93179995, 0.24696364, 0.9366]]
Epoch 001: Loss: 0.255, Accuracy: 93.278%, Val Loss: 0.242, Val Accuracy: 0.938
[[0, 0.2766259, 0.93179995, 0.24696364, 0.9366], [1, 0.2552488, 0.9327818, 0.24217199, 0.9376]]


KeyboardInterrupt: 

In [ ]:
seeds = np.arang(1,11)
for i, seed in enumerate(seeds):
    train("20220825_" + str(i+1), seed)